# <center> Explore and cluster the neighborhoods in Toronto</center>

## Part1

a) Code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes

In [1]:
# Import libraries need it for Part1
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
print('Libraries Panda and Numpy imported.')

Libraries Panda and Numpy imported.


In [2]:
# Just using Panda
GetTable = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
df_pc = GetTable[0]
# Fixing the misspelling - Wikipedia!, really?
df_pc.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df_pc.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
df_pc.shape

(288, 3)

In [3]:
# Using the BeautifulSoup package 
# Web scraping packages
from bs4 import BeautifulSoup
import requests
print('BeatifulSoup and Libraries Requests imported.')

BeatifulSoup and Libraries Requests imported.


In [4]:
# Load URL that need in the dataframe
Table_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(Table_url)
soup = BeautifulSoup(res.text, 'html.parser')
table = soup.find('table',{'class':'wikitable sortable'}).tbody
table_rows=table.find_all('tr')

pc_list = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        pc_list.append(row)

df_pcs = pd.DataFrame(pc_list, columns=["PostalCode", "Borough", "Neighborhood"])
print("Shape of Dataframe is - ",df_pcs.shape)
df_pcs.head()


Shape of Dataframe is -  (288, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


b) Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
df_pcs = df_pcs[df_pcs.Borough!="Not assigned"]
print("Shape of Dataframe is - ",df_pcs.shape)
df_pcs.head(11)

Shape of Dataframe is -  (211, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


c) If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [6]:
df_pcs['Neighborhood'] = np.where(df_pcs['Neighborhood'] == 'Not assigned', df_pcs['Borough'], df_pcs['Neighborhood'])
df_pcs.head(11)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [6]:
print("Shape of Dataframe is - ",df_pcs.shape)

Shape of Dataframe is -  (211, 3)


d) More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [7]:
df_pcs_grouped = pd.DataFrame(df_pcs.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index())
df_pcs_grouped.head(11)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [8]:
print("Shape of Dataframe with grouped Neiborhoods is - ",df_pcs_grouped.shape)

Shape of Dataframe with grouped Neiborhoods is -  (103, 3)


## Part 2

In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood

a) Installing GeoCoder

In [8]:
!conda install -c conda-forge geocoder --yes
print("Installation Done!")
import geocoder # import geocoder
print("Library GeoCoder imported!")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

In [9]:
# define geocoder function
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

In [10]:
# Add columns
df_pcs_grouped['Latitude'], df_pcs_grouped['Longitude'] = zip(*df_pcs_grouped['PostalCode'].apply(get_geocoder))
df_pcs_grouped.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [12]:
print("Shape of Dataframe with grouped Neiborhoods is - ",df_pcs_grouped.shape)

Shape of Dataframe with grouped Neiborhoods is -  (103, 5)


### End Part 2

## Part 3

### Explore and cluster the neighborhoods in Toronto

In [11]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium  # map rendering library

import json # library to handle JSON files
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
print('Libraries Json, Folium and Matplotlib imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  en

a) Map of Toronto

In [12]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto, Ontario are 43.653963, -79.387207.


In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(df_pcs_grouped['Latitude'], df_pcs_grouped['Longitude'], df_pcs_grouped['PostalCode'], df_pcs_grouped['Borough'], df_pcs_grouped['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

b) Studing the data

In [14]:
print('The dataframe has {} boroughs and {} PostalCodes.'.format(
        len(df_pcs_grouped['Borough'].unique()),
        df_pcs_grouped.shape[0]
    )
)

The dataframe has 11 boroughs and 103 PostalCodes.


c) Working only with Boroughs with the Toronto in the name

In [15]:
df_toronto = df_pcs_grouped[df_pcs_grouped['Borough'].str.contains('Toronto', regex=False)].reset_index(drop=True)
print("Shape of All Toronto Data dataframe is - ",df_toronto.shape)
print('The Toronto only dataframe has {} boroughs and {} PostalCodes.'.format(
        len(df_toronto['Borough'].unique()),
        len(df_toronto['PostalCode'].unique())
    )
)
df_toronto

Shape of All Toronto Data dataframe is -  (38, 5)
The Toronto only dataframe has 4 boroughs and 38 PostalCodes.


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676845,-79.295225
1,M4K,East Toronto,"The Danforth West,Riverdale",43.683262,-79.355120
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314673
3,M4M,East Toronto,Studio District,43.662766,-79.334830
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085
5,M4P,Central Toronto,Davisville North,43.712815,-79.388526
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
7,M4S,Central Toronto,Davisville,43.703395,-79.385964
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.690655,-79.383561
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686083,-79.402335


d) Mapping new Toronto only dataframe to see the difference

In [16]:
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, post, borough, neigh in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['PostalCode'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)
map_toronto2

e) Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [17]:
CLIENT_ID = 'IHLBQFARVGD0EWBHHMTKHMOYQSZHFFMCUWFC01QSANGENDX5' # your Foursquare ID
CLIENT_SECRET = 'RWVU1L0HQ0HOXYNBLWMCYST5NF5F3JFEG5IU4HRENDIPFHBL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IHLBQFARVGD0EWBHHMTKHMOYQSZHFFMCUWFC01QSANGENDX5
CLIENT_SECRET:RWVU1L0HQ0HOXYNBLWMCYST5NF5F3JFEG5IU4HRENDIPFHBL


f) Let's explore the first neighborhood in our dataframe.

In [18]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Neightborhood name: ' + df_toronto.loc[0, 'Neighborhood'])
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Neightborhood name: The Beaches
Latitude and longitude values of The Beaches are 43.67684518300007, -79.29522499999996.


g) Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [19]:
# type your answer here
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=IHLBQFARVGD0EWBHHMTKHMOYQSZHFFMCUWFC01QSANGENDX5&client_secret=RWVU1L0HQ0HOXYNBLWMCYST5NF5F3JFEG5IU4HRENDIPFHBL&ll=43.67684518300007,-79.29522499999996&v=20180605&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Dip 'n Sip,Coffee Shop,43.678897,-79.297745
4,Upper Beaches,Neighborhood,43.680563,-79.292869


h) Explore all the Neighborhood with Borough name like 'Torornto', by PostalCode

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
# postcode=df_toronto['PostalCode'], br=df_toronto['Borough'], 
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
print('Shape of Toronto venues: ', toronto_venues.shape)

toronto_venues.head()

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676845,-79.295225,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676845,-79.295225,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676845,-79.295225,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676845,-79.295225,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
4,The Beaches,43.676845,-79.295225,Upper Beaches,43.680563,-79.292869,Neighborhood


In [22]:
print('The Toronto venues dataframe has {} unique Neighborhood.'.format(
        len(toronto_venues['Neighborhood'].unique())
    )
)

The Toronto venues dataframe has 37 unique Neighborhood.


i) How many venues for each neightborhood set

In [23]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,64,64,64,64,64,64
"Brockton,Exhibition Place,Parkdale Village",70,70,70,70,70,70
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",73,73,73,73,73,73
"Cabbagetown,St. James Town",41,41,41,41,41,41
Central Bay Street,98,98,98,98,98,98
"Chinatown,Grange Park,Kensington Market",98,98,98,98,98,98
Christie,9,9,9,9,9,9


i) Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 212 uniques categories.


j)  Analyze Each Neighborhood

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [26]:
print("Shape of Dataframe for Toronto Neiborhoods Venues is - ",toronto_onehot.shape)

Shape of Dataframe for Toronto Neiborhoods Venues is -  (1748, 212)


k) Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category so we can identify the most common, see new size

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print("Shape of Dataframe for Toronto Neiborhoods Grouped by Venues is - ",toronto_grouped.shape)
toronto_grouped.head()

Shape of Dataframe for Toronto Neiborhoods Grouped by Venues is -  (37, 212)


,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.0,0.03,0.0,0.010000,0.0,0.000000,0.030000,0.0,0.000000,0.0,0.000000,0.020000,0.0,0.030000,0.000000,0.010000,0.0,0.000000,0.02,0.0,0.01,0.020000,0.000000,0.0,0.01,0.030000,0.01,0.0,0.000000,0.070000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.090000,0.0,0.000000,0.0,0.01,0.000000,0.0,0.020000,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.03,0.01,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.010000,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.01,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.03,0.0,0.0,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.020000,0.010000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.050000,0.000000,0.0,0.010000,0.00,0.000000,0.000000,0.010000,0.030000,0.000000,0.0,0.0,0.00,0.0,0.0,0.010000,0.0,0.0,0.0,0.000000,0.010000,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.01,0.00,0.000000,0.0,0.0,0.010000,0.000000,0.01,0.000000,0.01,0.0,0.0,0.0,0.0,

l) Sort the venues in descending order and put results in a dataframe for top 10 by Neighborhood

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Bar,Japanese Restaurant,Steakhouse,Asian Restaurant,Gastropub,Restaurant,Deli / Bodega
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Farmers Market,Beer Bar,Hotel,Steakhouse,Seafood Restaurant,Cheese Shop
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Restaurant,Furniture / Home Store,Sandwich Place,Vegetarian / Vegan Restaurant,Beer Bar,Supermarket,Italian Restaurant,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Bar,Café,Steakhouse,Hotel,Italian Restaurant,American Restaurant,Pizza Place,Thai Restaurant,Sushi Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Italian Restaurant,Gym / Fitness Center,Café,Restaurant,Bar,Sandwich Place,Speakeasy,Pub,Park


#### Looks like Coffee Shops are very populer

### Cluster Neighborhoods

m) Run *k*-means to cluster the neighborhood into 5 clusters, and create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [63]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [64]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676845,-79.295225,2.0,Health Food Store,Trail,Pub,Coffee Shop,Wings Joint,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.683262,-79.355120,0.0,Park,Bus Line,Discount Store,Grocery Store,Fast Food Restaurant,Wings Joint,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314673,2.0,Pizza Place,Burrito Place,Movie Theater,Liquor Store,Burger Joint,Sandwich Place,Steakhouse,Sushi Restaurant,Pub,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.662766,-79.334830,2.0,Bakery,Diner,Italian Restaurant,Café,Coffee Shop,American Restaurant,Brewery,Arts & Crafts Store,Sushi Restaurant,Bar
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,1.0,Bus Line,Swim School,Wings Joint,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


n) let's visualize the resulting clusters

In [65]:
# Delete Roselawn that doesn't have venues data or cluster
toronto_merged = toronto_merged[toronto_merged.Neighborhood!="Roselawn"]
toronto_merged.reset_index(drop=True,inplace=True)
print("Shape of toronto_merged Dataframe is - ",toronto_merged.shape)
# Change stype in Custer Labels
toronto_merged['Cluster Labels']= toronto_merged['Cluster Labels'].apply(np.int64)
print(toronto_merged['Cluster Labels'].dtypes)

Shape of toronto_merged Dataframe is -  (37, 16)
int64


In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

m) Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,"The Danforth West,Riverdale",0,Park,Bus Line,Discount Store,Grocery Store,Fast Food Restaurant,Wings Joint,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market
6,Central Toronto,North Toronto West,0,Park,Playground,Gym Pool,Garden,Wings Joint,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
8,Central Toronto,"Moore Park,Summerhill East",0,Park,Playground,Convenience Store,Gym,Restaurant,Department Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
10,Downtown Toronto,Rosedale,0,Park,Playground,Building,Gym / Fitness Center,Bank,Department Store,Ethiopian Restaurant,Deli / Bodega,Flea Market,Fish Market
33,West Toronto,"High Park,The Junction South",0,Park,Dive Bar,Wings Joint,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


Cluster 1 = Parks

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + [2]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,Lawrence Park,1,Bus Line,Swim School,Wings Joint,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


Cluster 2 = Bus Line

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,2,Health Food Store,Trail,Pub,Coffee Shop,Wings Joint,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
2,East Toronto,"The Beaches West,India Bazaar",2,Pizza Place,Burrito Place,Movie Theater,Liquor Store,Burger Joint,Sandwich Place,Steakhouse,Sushi Restaurant,Pub,Fish & Chips Shop
3,East Toronto,Studio District,2,Bakery,Diner,Italian Restaurant,Café,Coffee Shop,American Restaurant,Brewery,Arts & Crafts Store,Sushi Restaurant,Bar
5,Central Toronto,Davisville North,2,Food & Drink Shop,Park,Hotel,Gym,Breakfast Spot,Dog Run,Clothing Store,Event Space,Flower Shop,Flea Market
7,Central Toronto,Davisville,2,Dessert Shop,Italian Restaurant,Café,Sandwich Place,Park,Coffee Shop,Thai Restaurant,Pizza Place,Seafood Restaurant,Farmers Market
9,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",2,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Wings Joint,Event Space,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
11,Downtown Toronto,"Cabbagetown,St. James Town",2,Park,Coffee Shop,Italian Restaurant,Bakery,Café,Pizza Place,Restaurant,Jewelry Store,Snack Place,Deli / Bodega
12,Downtown Toronto,Church and Wellesley,2,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Sushi Restaurant,Dance Studio,Burger Joint,Pub,Hotel,Fast Food Restaurant
13,Downtown Toronto,"Harbourfront,Regent Park",2,Coffee Shop,Restaurant,Breakfast Spot,Italian Restaurant,Mexican Restaurant,Electronics Store,Bakery,Event Space,Thai Restaurant,Theater
14,Downtown Toronto,"Ryerson,Garden District",2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Ramen Restaurant,Tea Room,Bookstore,Plaza,Hotel


Cluster 3 = Coffee Shop

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",3,Harbor / Marina,Music Venue,Café,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


Cluster 4 = Harbor/Marina

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,"Forest Hill North,Forest Hill West",4,Park,Lawyer,Wings Joint,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


Cluster 5 = Lawyer

In [74]:
# Define a function to map the values 
def set_value(row_number, assigned_value): 
    return assigned_value[row_number] 
  
# Create the dictionary 
cluster_dictionary ={0 : 'Park', 1 : 'Bus Line', 2 : 'Coffee Shop', 3 : 'Harbor/Marina', 4 : 'Lawyer'} 
  
# Add a new column named 'Price' 
toronto_merged['Cluster Name'] = toronto_merged['Cluster Labels'].apply(set_value, args =(cluster_dictionary, )) 
  
# Print the DataFrame 
toronto_merged. head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Name
0,M4E,East Toronto,The Beaches,43.676845,-79.295225,2,Health Food Store,Trail,Pub,Coffee Shop,Wings Joint,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Coffee Shop
1,M4K,East Toronto,"The Danforth West,Riverdale",43.683262,-79.355120,0,Park,Bus Line,Discount Store,Grocery Store,Fast Food Restaurant,Wings Joint,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Park
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314673,2,Pizza Place,Burrito Place,Movie Theater,Liquor Store,Burger Joint,Sandwich Place,Steakhouse,Sushi Restaurant,Pub,Fish & Chips Shop,Coffee Shop
3,M4M,East Toronto,Studio District,43.662766,-79.334830,2,Bakery,Diner,Italian Restaurant,Café,Coffee Shop,American Restaurant,Brewery,Arts & Crafts Store,Sushi Restaurant,Bar,Coffee Shop
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,1,Bus Line,Swim School,Wings Joint,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Bus Line


## Finding: Most Neighborhoods have Coffee Shops  in the top 10 venues categories